In [26]:
from Dataset import lerCSV,lerXLSX
import warnings
import sys 
import numpy as np
import pickle
import theanets
from sklearn import preprocessing
from random import shuffle
from sklearn.metrics import precision_score,recall_score,log_loss
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from math import sqrt
import math
from roc import RocAucScoreOp
%matplotlib inline

warnings.filterwarnings("ignore") 

In [27]:
  
matrix = lerXLSX("Banco de Dados - Infarto treinoTeste.xlsx",training=True,agrupamento=False)
matrixValidacao = lerXLSX("Banco de Dados - Infarto validacao.xlsx",training=True,agrupamento=False)


def specificity(outputs,predicts):
    neg = np.sum([output == 0 for output in outputs])
    tn = np.sum([predicts[x] == outputs[x] and outputs[x] == 0 for x in range(len(predicts))])
    return tn / neg

## Normalizando o Dataset

In [28]:
min_max_scaler = preprocessing.MinMaxScaler()
matrixProcessed = min_max_scaler.fit_transform(matrix[:,:3])

min_max_scalerV = preprocessing.MinMaxScaler()
matrixProcessedValidacao = min_max_scalerV.fit_transform(matrixValidacao[:,:3])

## Criando O Modelo e Treinando

In [30]:
metric =  ["neural network (3,2)",[3, 2]]
inputs = matrixProcessed
weights = np.array([int(matrix[y,3] == 1)*2+1 for y in range(len(matrix[:,2]))],dtype="int32")        
outputs = np.array([int(matrix[y,3] == 1) for y in range(len(matrix[:,2]))],dtype="int32")

model = theanets.Classifier(metric[1],loss='CrossEntropy',weighted=True)  
saidaT = model.train([inputs, outputs,weights],algo='sgd',learning_rate=0.01,momentum=0.9)

## Validando

In [31]:
saidaValidacao  = np.array([int(matrixValidacao[y,3] == 1) for y in range(len(matrixValidacao[:,2]))],dtype="int32")
predicts = model.predict(matrixProcessedValidacao)
predictsProbas = model.predict_proba(matrixProcessedValidacao)
score = log_loss(saidaValidacao, predictsProbas)
recall = recall_score(saidaValidacao, predicts)
precision = specificity(saidaValidacao, predicts)

print(""+ metric[0] + " = Error:%2.2e"%(score))
print(""+ metric[0] + " = Sensitive:%2.2e"%(recall))
print(""+ metric[0] + " = specificity:%2.2e"%(precision))    

neural network (3,2) = Error:6.32e-01
neural network (3,2) = Sensitive:9.09e-01
neural network (3,2) = specificity:6.06e-01
